In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

# קריאת הקובץ עם טיפול בערכים מעורבים
file_path = 'flights_cleaned.csv'  # ודא שהשם של הקובץ נכון
data = pd.read_csv(file_path, dtype={'ORIGIN_AIRPORT': str, 'DESTINATION_AIRPORT': str}, low_memory=False)

# יצירת גרף מכוון מהנתונים
def create_directed_graph(data):
    G = nx.DiGraph()
    for index, row in data.iterrows():
        G.add_edge(row['ORIGIN_AIRPORT'], row['DESTINATION_AIRPORT'])
    return G

graph = create_directed_graph(data)

# חישוב Betweenness Centrality
betweenness = nx.betweenness_centrality(graph)

# יצירת DataFrame עבור מדדי המרכזיות
centrality_df = pd.DataFrame(list(betweenness.items()), columns=['Airport', 'Betweenness_Centrality'])

# חישוב עיכובים ממוצעים לכל נמל תעופה
delay_data = data.groupby('ORIGIN_AIRPORT')['DEPARTURE_DELAY'].mean().reset_index()
delay_data.columns = ['Airport', 'Average_Departure_Delay']

# מיזוג הנתונים
merged_data = pd.merge(centrality_df, delay_data, on='Airport')

# הדפסת התוצאות
print(merged_data)

# יצירת גרף להשוואת מדדי המרכזיות לעיכובים הממוצעים
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Betweenness_Centrality', y='Average_Departure_Delay', data=merged_data)
plt.xlabel('Betweenness Centrality')
plt.ylabel('Average Departure Delay')
plt.title('Betweenness Centrality vs. Average Departure Delay')
plt.show()

# חישוב הקורלציה
correlation = merged_data['Betweenness_Centrality'].corr(merged_data['Average_Departure_Delay'])
print(f'Correlation between Betweenness Centrality and Average Departure Delay: {correlation}')